In [1]:
import numpy as np
from types import SimpleNamespace
%load_ext autoreload
%autoreload 2
from copy import deepcopy
import time
from scipy import optimize
from scipy import interpolate
import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})

# Dynamic Optimization Problem: Dentists

We start of by defining a set of parameters and then build our cost function that our agent is facing when deciding on going to the dentist or not. We plot our cost function. 
The basic idea is that it costs to go to the dentist every time you go, but if you postpone your check ups the total cost will increase.

In [2]:
par = SimpleNamespace()
par.x = 0 #x being the level of usage of your teeth
par.FC = 10 #Fixed cost of going to the dentist
par.MC = 2 #Marginal cost of going to the dentist, depends on your usage of the teeth (when you last went)
par.beta = 0.9 #Discount factor
par.exponent = 1.5 #How bad your teeth decay if you don't see a dentist
par.gamma = 0.1
par.T = 3
par.boost = 5
par.threshold = 15 #threshold level to account for the severity of toothache  
par.weight = 1.3 #weight of toothache severity relative to the disutility caused by tooth decay

Defining a cost function (cost_dentist) that calculates the cost of going to the dentist or not, based on input parameters.

In [3]:
def cost_dentist(par, d, x):
    if d == 0: 
        return  0 #cost of not going to the dentist 
    if d == 1: 
        return par.FC + par.MC*x #cost of going to the dentist 

Defining a function (teeth_decay) that reflects how a persons teeth transition over time based on input parameters.

In [4]:
def teeth_decay(par, d, x_t): #How your teeth are transitioning from one time period to the next
    if d == 0:
        
        x_1 = int(x_t**par.exponent) + 3 #Decay of teeth of not going based on current state (x_t), an exponent and a constant (3) 
    else:
        x_1 = max(x_t - par.boost, 0) #improvement of going to the dentist (current state (x_t) or a maximum of a reset stage of 0)
    return x_1

Defining a function (disutil_ache) that calculates the disutility caused by toothache based on a persons current teeth state.

In [5]:
def disutil_ache(par, x_t): 
    disutil = (x_t**(1-par.gamma) - 1) / (1 - par.gamma)
    return int(disutil)

Defining a function (disutil_ache_threshold) that calculates the disutility caused by toothache based on a persons current teeth state in relation to a threshold level

In [6]:
def disutil_ache_threshold(par, x_t): 
    if x_t >= par.threshold:
        disutil = (par.weight * (x_t - par.threshold))**(1 - par.gamma) / (1 - par.gamma)
    else:
        disutil = (x_t**(1-par.gamma) - 1) / (1 - par.gamma)
    return int(disutil)

In [ ]:
def total_util(par, d, x_t):
    return -cost_dentist(par, d, x_t) - disutil_ache(par, x_t)

In [ ]:
c_going = []
expo = []
logo = []
for i in range(0,50):
    c_going.append(cost_dentist(par,1, i))#cost of not going for all diff. stages
    expo.append(i**1.5)#incorporate a decay rate of not going
    logo.append(np.log(i+1))
plt.plot(c_going)
plt.plot(logo)
plt.plot(expo)
plt.title('Cost of going to the dentist for different usage levels of teeth')
plt.show

# Version 2

In [ ]:
import numpy as np
from collections import namedtuple
import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})

# Define the parameters for the dentist model
DentistParams = namedtuple('DentistParams', ['FC', 'MC', 'alpha', 'beta', 'gamma', 'pr', 'budget'])
par = DentistParams(FC=10, MC=2, alpha=0.5, beta=0.95, gamma=0.5, pr=0.8, budget=20)

# Define the initial conditions for the model
x0 = 0
T = 5

# Solve the dentist model and print the results
x, c, u, cost, discounted_utility, d, total_discounted_utility, total_cost = solve_dentist_model(par, x0, T)
print("x: ", x)
print("c: ", c)
print("u: ", u)
print("cost: ", cost)
print("discounted_utility: ", discounted_utility)
print("d: ", d)
print("total_discounted_utility: ", total_discounted_utility)
print("total_cost: ", total_cost)
plt.plot(u)
plt.plot(x)
plt.plot(c)
plt.plot(d)
plt.plot(cost)
plt.plot(discounted_utility)
plt.plot(total_discounted_utility)
plt.plot(total_cost)


In [ ]:
par = SimpleNamespace()
par.x = 0 #x being the level of usage of your teeth
par.FC = 10 #Fixed cost of going to the dentist
par.MC = 2 #Marginal cost of going to the dentist, depends on your usage of the teeth (when you last went)
par.alpha = 0.5#parameter in the utility function 
par.beta = 0.9 #Discount factor
par.gamma = 1.5 #How bad your teeth decay if you don't see a dentist
par.pr = 0.1 #the probability of needing a filling 


In [ ]:
def solve_dentist_model(par, x, T):
    # Initialize arrays to store results
    x = np.zeros(T+1)
    c = np.zeros(T+1)
    u = np.zeros(T+1)
    cost = np.zeros(T+1)
    discounted_utility = np.zeros(T+1)
    d = np.zeros(T+1, dtype=int)

    # Set initial values
    x[0] = 0
    c[0] = 0
    u[0] = 0
    cost[0] = 0
    discounted_utility[0] = 0
    d[0] = 0

    for t in range(1, T+1):
        # Compute costs and benefits of going to the dentist and not going
        cost_dentist = par.FC + par.MC * x[t-1]
        discounted_utility_dentist = (par.alpha * x[t-1]**par.gamma) / (1 + par.beta)
        utility_no_dentist = (x[t-1]**par.gamma) / (1 + par.beta**(t-1))

        # Compute expected utility of going to the dentist and not going
        expected_utility_dentist = par.pr * discounted_utility_dentist + (1 - par.pr) * utility_no_dentist
        expected_utility_no_dentist = utility_no_dentist

        # Choose action that maximizes expected utility
        if expected_utility_dentist >= expected_utility_no_dentist and cost_dentist <= par.budget:
            x[t] = x[t-1] + 1
            c[t] = cost_dentist
            u[t] = discounted_utility_dentist
            cost[t] = cost_dentist
            discounted_utility[t] = discounted_utility_dentist
            d[t] = 1
        else:
            x[t] = x[t-1]
            c[t] = 0
            u[t] = utility_no_dentist
            cost[t] = 0
            discounted_utility[t] = utility_no_dentist
            d[t] = 0
    
            

    # Compute total discounted utility and cost
    total_discounted_utility = np.sum(discounted_utility)
    total_cost = np.sum(cost)

    # Return results
    return x, c, u, cost, discounted_utility, d, total_discounted_utility, total_cost



In [ ]:
# Print the results
print("x:", x)
print("c:", c)
print("u:", u)
print("cost:", cost)
print("discounted_utility:", discounted_utility)
print("d:", d)
print("total_discounted_utility:", total_discounted_utility)
print("total_cost:", total_cost)

### Possible extensions: 

° Income and savings 

° Life style choices (healthy or unhealthy) with different impacts on x.

° Varying MC and FC

° Age impacting decay of teeth. Say after 50 years, consequences of not going (increase in x if d=0) goes up. Or marginal cost could double.

° Implementing a subsidy of one dollar. Does it make more sense to subsidize the marginal or the fixed cost. 